In [8]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
#%matplotlib nbagg
#%matplotlib notebook
#%matplotlib qt
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [89]:
#results = np.load("runhet_recon_sweep_score_mat.npy")
#results = np.load("runp_1_2_sweep_dp_score_mat.npy")
#results = np.load("runp_1_2_sweep_eo_041719_prod_score_mat.npy")
results0 = np.load("results/runorfunc_051419_0_score_mat.npy")#[:,:-1]
results1 = np.load("results/runorfunc_051419_1_score_mat.npy")#[:,:-1]
results2 = np.load("results/runorfunc_051419_2_score_mat.npy")#[:,:-1]
results3 = np.load("results/runorfunc_051419_3_score_mat.npy")#[:,:-1]
results4 = np.load("results/runorfunc_051419_4_score_mat.npy")#[:,:-1]

In [90]:
#results = results[:, :]
results = (results0 + results1 + results2 + results3 + results4) / 5
results

array([[0.40945514, 0.41454328, 0.3736779 , 0.3741186 , 0.31923078,
        0.30560898, 0.28581732, 0.27652242],
       [0.2771234 , 0.28076922, 0.27299678, 0.28129006, 0.19491186,
        0.20228364, 0.16534456, 0.1785657 ],
       [0.22688304, 0.24102564, 0.2118189 , 0.20600962, 0.17776442,
        0.20777244, 0.17708332, 0.19082532],
       [0.17572114, 0.17884616, 0.18745994, 0.18325322, 0.16658654,
        0.16149838, 0.1741186 , 0.1941907 ],
       [0.17844552, 0.18221154, 0.17127406, 0.18205126, 0.18120994,
        0.16458332, 0.15985578, 0.19535258],
       [0.18337342, 0.16794872, 0.19447114, 0.18513622, 0.18697918,
        0.17099356, 0.1746795 , 0.17860578],
       [0.20641026, 0.18934296, 0.18149038, 0.18289262, 0.18842148,
        0.1810096 , 0.178125  , 0.18946314]])

In [91]:
#coeffs = np.array(['0_0', '0_005', '0_01', '0_05', '0_1', '0_2', '0_5', '1_0', '2_0', '4_0', '6_0', '10_0'])
#coeffs = np.array([0. , 0.005, 0.01, 0.05, 0.1, 0.2, 0.5, 1., 2., 4., 6., 10.])
#xcoeffs = [0.0, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.5, 1.0, 4.0]
#ycoeffs = [0.0, 0.001, 0.005, 0.01] #, 0.03]#, 0.05, 0.1, 0.15, 0.2, 0.3]
#xcoeffs = [0.0, 0.1, 0.5, 1.0, 3.0, 5.0, 10.0, 15.0]
#ycoeffs = [0.0, 0.0005, 0.001, 0.002, 0.003, 0.005, 0.007, 0.01, 0.03, 0.05, 0.1] #, 0.7]
#xcoeffs = [0.0, 0.1, 0.5, 1.0, 3.0, 5.0, 10.0, 15.0]
#ycoeffs = [0.0, 0.001, 0.005, 0.01, 0.025, 0.05, 0.075, 0.1, 0.3, 0.5]
#n = len(coeffs)

xcoeffs = [0.0, 0.5, 1.0, 3.0, 5.0, 10.0, 15.0]
ycoeffs = [0.0, 0.002, 0.005, 0.01, 0.03, 0.05, 0.07, 0.1]



In [92]:
X = np.repeat(xcoeffs, len(ycoeffs)).reshape((len(xcoeffs), len(ycoeffs)))
Y = np.repeat(ycoeffs, len(xcoeffs)).reshape((len(xcoeffs), len(ycoeffs)), order='F')
print(X)
print(Y)

[[ 0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5]
 [ 1.   1.   1.   1.   1.   1.   1.   1. ]
 [ 3.   3.   3.   3.   3.   3.   3.   3. ]
 [ 5.   5.   5.   5.   5.   5.   5.   5. ]
 [10.  10.  10.  10.  10.  10.  10.  10. ]
 [15.  15.  15.  15.  15.  15.  15.  15. ]]
[[0.    0.002 0.005 0.01  0.03  0.05  0.07  0.1  ]
 [0.    0.002 0.005 0.01  0.03  0.05  0.07  0.1  ]
 [0.    0.002 0.005 0.01  0.03  0.05  0.07  0.1  ]
 [0.    0.002 0.005 0.01  0.03  0.05  0.07  0.1  ]
 [0.    0.002 0.005 0.01  0.03  0.05  0.07  0.1  ]
 [0.    0.002 0.005 0.01  0.03  0.05  0.07  0.1  ]
 [0.    0.002 0.005 0.01  0.03  0.05  0.07  0.1  ]]


In [93]:
xmesh = np.linspace(xcoeffs[0], xcoeffs[-1])
ymesh = np.linspace(ycoeffs[0], ycoeffs[-1])
xmeshgrid = np.repeat(xmesh, len(ymesh)).reshape((len(xmesh), len(ymesh)))
ymeshgrid = np.repeat(ymesh, len(xmesh)).reshape((len(xmesh), len(ymesh)), order='F')
#xmeshgrid
#ymeshgrid

logreg = np.ones(xmeshgrid.shape) * 0.238 #0.16500 #0.132 #0.0464 #0.1872 #0.0706 #0.2004 #0.0536 #0.1932


In [94]:
fig = plt.figure(figsize = (10, 6))
ax = fig.gca(projection='3d')
surf = ax.plot_surface(X, Y, results, cmap=cm.coolwarm, linewidth=0, antialiased=False, edgecolor='black')
plane = ax.plot_wireframe(xmeshgrid, ymeshgrid, logreg, color='green', linewidth=.5, antialiased=False)
#surf.set_label('Multi-Study LAFTR')
plane.set_label('Logistic regression baseline')
plt.xlabel('Fairness coefficient')
plt.ylabel('Reconstruction coefficient')
plt.title('OR function dataset 05/14/19')
ax.set_zlabel('Error')
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.legend(loc='lower right')
plt.show()